In [1]:
scop_url = 'scop.txt'
ids = []
folds = []
with open(scop_url, 'r') as f:
    for line in f:
        l = line.split(' ')
        pdb_id = l[1]
        ids.append(pdb_id)
        fold = l[-1].split('CF=')[-1].split(',SF')[0]
        folds.append(fold)
        id_dict = {l.split(' ')[1]:l.split(' ')[-1].split('CF=')[-1].split(',SF')[0] for l in f if l[0] != '#'}

pdb_ids = list(id_dict.keys())
folds = list(id_dict.values())

In [2]:
import pandas as pd

def countlist(l):
    counts = dict()
    for i in l:
        counts[i] = counts.get(i, 0) + 1
    return counts
def getsheet(xl, n):
    return xl.parse(xl.sheet_names[n])

#use to get a list of all the values in a columns across ALL sheets
def getcol(xl, c):
    clist = []
    for s in xl.sheet_names:
        try:
            clist.extend(xl.parse(s)[c])
        except Exception as e:
            print(e)
    return clist

fam_url = 'families.xlsx'
fam_xl = pd.ExcelFile(fam_url)

In [17]:
fam_ids = getcol(fam_xl, 'id')
vdict= {i:id_dict[i.upper()] for i in fam_ids if i.upper() in pdb_ids}
des_file = 'scop-des.txt'
with open(des_file, 'r') as f:
    des_dict = {l.split(' ')[0]: l.split(' ', 1)[1] for l in f if l[0]!='#'}

v_des = {k:des_dict[v] for k, v in vdict.items()}
countlist(v_des.values())

{'Nucleoplasmin-like/VP (viral coat and capsid proteins)\n': 194,
 'Viral glycoprotein  central and dimerisation domains\n': 1,
 'Hepatitis B viral capsid (hbcag)\n': 5,
 'RNA bacteriophage capsid protein\n': 4,
 'Soc-like\n': 1,
 'Phage major capsid protein-like\n': 6,
 'CoA-dependent acyltransferases\n': 1,
 'LDL receptor-like module\n': 1,
 'Reovirus inner layer core protein p3\n': 4,
 'A virus capsid protein alpha-helical domain\n': 1,
 'Trypsin-type beta(6)-barrel\n': 5,
 'TNF-like\n': 4,
 'L-A virus major coat protein\n': 1}

In [21]:
id_des = {k:des_dict[v] for k,v in id_dict.items() if v in des_dict.keys()}

data_url = 'PhrenologyData.xlsx'
data_xl = pd.ExcelFile(data_url)

pdbs = getcol(data_xl, 'pdb')

pdb_folds = {i:id_des[i.upper()] if i.upper() in id_des.keys() else 'Unidentified' for i in pdbs}

df = pd.read_excel(data_url)

fold_df = pd.DataFrame(pdb_folds.values(), columns=['fold'], index=None)

full_df = pd.concat((df, fold_df), axis=1)

full_df.to_excel('phrenology_folds.xlsx', index=False)

In [47]:
#get all viper ids not registered in scop

import requests
import json
import pandas as pd
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import re
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook

disable_warnings(InsecureRequestWarning)

base_url = "https://viperdb.scripps.edu"

all_tnums_url = "/services/tnumber_index.php?serviceName=tnumbers"
all_genuses_url = "/services/genus_index.php?serviceName=genus"
all_families_url = "/services/family_index.php?serviceName=families"

tnum_members_url = "/services/tnumber_index.php?serviceName=tnumber_members&tnumber="
genus_members_url = "/services/genus_index.php?serviceName=genus_members&genus="
family_members_url = "/services/family_index.php?serviceName=family_members&family="

rcsb_url = 'https://data.rcsb.org/rest/v1/core/entry/'
vdb_url = "https://viperdb.scripps.edu/"
radius_url = "services/biodata.php?serviceName=radius_formatted&VDB="

def getrcsb(c):
    res = requests.get(rcsb_url + c, verify=False)
    return res
def getrad(c):
    res = requests.get(vdb_url + radius_url + c, verify=False)
    return res

def tnum_url(n):
    return base_url + tnum_members_url + str(n)
def genus_url(n):
    return base_url + genus_members_url + str(n)
def family_url(n):
    return base_url + family_members_url + str(n)

#sort list of dictionaries by resolution and take the n lowest
def lowres_dictlist(dictlist, n):
    lowres = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        lowres[k] = sortres[:n]
    return lowres
def dictlist(dictlist):
    res = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        res[k] = sortres
    return res

tnums_res = requests.get((base_url + all_tnums_url), verify=False)
tnums = json.loads(tnums_res.text)

tnum_members = {}
for i in range(len(tnums)):
    tnum = tnums[i]["tnumber"]
    members_res = requests.get(tnum_url(tnum), verify=False)
    tnum_members[tnum] = json.loads(members_res.text)

viper_ids = list(set([item['entry_id'] for sublist in tnum_members.values() for item in sublist]))

ks_ins =  [k.casefold() for k in id_dict.keys()]
un_ids = {'pdb_id': [i.upper() for i in viper_ids if i.casefold() not in ks_ins]}
df = pd.DataFrame(data=un_ids, index=None)
df.to_excel('unidentified_ids.xlsx', index=False)